In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing import image

In [3]:
import os
import shutil
import random

def create_split_folders(base_path, train_size=0.7, valid_size=0.15, test_size=0.15):
    """
    Split dataset into train, validation, and test sets.
    
    Parameters:
        base_path (str): The base path to the dataset.
        train_size (float): Proportion of data to be used for training.
        valid_size (float): Proportion of data to be used for validation.
        test_size (float): Proportion of data to be used for testing.
    """
    assert (train_size + valid_size + test_size) == 1.0, "Sizes must sum to 1."

    # Define directories
    train_dir = os.path.join(base_path, 'train')
    valid_dir = os.path.join(base_path, 'valid')
    test_dir = os.path.join(base_path, 'test')

    # Create directories if they do not exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # List all plant categories (subdirectories)
    plant_categories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

    for category in plant_categories:
        category_path = os.path.join(base_path, category)

        # Create corresponding category directories in train, valid, and test folders
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(valid_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        # List all images in the current category
        all_files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        random.shuffle(all_files)  # Shuffle the list to ensure random distribution

        # Calculate split indices
        total_files = len(all_files)
        train_end = int(train_size * total_files)
        valid_end = train_end + int(valid_size * total_files)

        # Split files
        train_files = all_files[:train_end]
        valid_files = all_files[train_end:valid_end]
        test_files = all_files[valid_end:]

        def move_files(file_list, target_dir):
            for file_name in file_list:
                src_path = os.path.join(category_path, file_name)
                dst_path = os.path.join(target_dir, file_name)
                shutil.move(src_path, dst_path)

        # Move files to corresponding directories
        move_files(train_files, os.path.join(train_dir, category))
        move_files(valid_files, os.path.join(valid_dir, category))
        move_files(test_files, os.path.join(test_dir, category))

# Define the base path to your dataset
base_path = 'medicinalplant'  # Update this path
create_split_folders(base_path)


In [6]:
# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2, rotation_range=20)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    'dataset/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
# Obtain class indices
class_indices = train_generator.class_indices
# Create a mapping from index to class name
index_to_class = {int(v): k for k, v in class_indices.items()}

Found 15750 images belonging to 45 classes.
Found 3375 images belonging to 45 classes.


In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # Adjust number of classes

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Compile the model with the correct parameter
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary to confirm
print(model.summary())


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,696,138 (98.02 MB)

 Trainable params: 21,561,354 (82.25 MB)

 Non-trainable params: 4,134,784 (15.77 MB)

None


In [10]:
# Recompile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
from tensorflow.keras.optimizers import Adam

# Fit the model
model.fit(
    train_generator,
    epochs=1,
    validation_data=valid_generator
)


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 45), output.shape=(None, 10)

In [12]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(45, activation='softmax')(x)  # Adjust number of classes to 45

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary to confirm changes
print(model.summary())


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,732,013 (98.16 MB)

 Trainable params: 21,597,229 (82.39 MB)

 Non-trainable params: 4,134,784 (15.77 MB)

None


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming you have defined train_directory and valid_directory
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # Other augmentations if needed
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Should match the number of classes
)

valid_generator = valid_datagen.flow_from_directory(
    'dataset/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Should match the number of classes
)

print(train_generator.class_indices)  # Verify the number of classes


Found 15750 images belonging to 45 classes.
Found 3375 images belonging to 45 classes.
{'Aloevera': 0, 'Amla': 1, 'Arive-Dantu': 2, 'Basale': 3, 'Betel': 4, 'Bhrami': 5, 'Coriender': 6, 'Crape_Jasmine': 7, 'Curry': 8, 'Drumstick': 9, 'Fenugreek': 10, 'Guava': 11, 'Henna': 12, 'Hibiscus': 13, 'Honge': 14, 'Indian_Beech': 15, 'Indian_Mustard': 16, 'Insulin': 17, 'Jackfruit': 18, 'Jamaica_Cherry-Gasagase': 19, 'Jamun': 20, 'Jasmine': 21, 'Karanda': 22, 'Lemon': 23, 'Mango': 24, 'Marigold': 25, 'Mexican_Mint': 26, 'Mint': 27, 'Neem': 28, 'Oleander': 29, 'Palak(Spinach)': 30, 'Papaya': 31, 'Parijata': 32, 'Peepal': 33, 'Pomegranate': 34, 'Rasna': 35, 'Rose': 36, 'Rose_apple': 37, 'Roxburgh_fig': 38, 'Sandalwood': 39, 'Seethapala': 40, 'Tamarind': 41, 'Tulsi': 42, 'Turmeric': 43, 'ashoka': 44}


In [15]:
from tensorflow.keras.optimizers import Adam

# Recompile the model
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Fit the model
model.fit(
    train_generator,
    epochs=1,
    validation_data=valid_generator
)


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


493/493 ━━━━━━━━━━━━━━━━━━━━ 7653s 16s/step - accuracy: 0.4111 - loss: 2.0231 - val_accuracy: 0.6551 - val_loss: 1.1278


In [16]:
model.save('model/model_0.h5')

In [17]:
# Evaluate the model
test_loss, test_acc = model.evaluate(valid_generator)
print(f'Test accuracy: {test_acc * 100:.2f}%')

106/106 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.6445 - loss: 1.1727
Test accuracy: 65.51%


In [12]:
index_to_class = {0:'Aloevera',1:'Amla',2:'Arive-Dantu',3:'ashoka',4:'Basale',5:'Betel',
                  6:'Bhrami',7:'Coriender',8:'Crape_jasmine',9:'Curry',10:'Drumstick',
                  11:'Fenugreek',12:'Guava',13:'Henna',14:'Hibiscus',15:'Honge',16:'Indian_Beech',
                  17:'Indian_Mustard',18:'Insulin',19:'Jackfruit',20:'Jamaica_Cherry-Gasagase',
                  21:'Jamun',22:'Jasmin',23:'Karanda',24:'Lemon',25:'Mango',26:'Marigold',
                  27:'Mexican_Mint',28:'Mint',29:'Neem',30:'Oleander',31:'Palak(Spinach)',
                  32:'Papaya',33:'Parijata',34:'Peepal',35:'Pomegranate',36:'Rasna',37:'Rose',
                  38:'Rose_apple',39:'Roxburgh_fig',40:'Sandalwood',41:'Seethapala',42:'tamarind',
                  43:'Tulsi',44:'Turmeric'}

## Function to predict plant disease for a new image with error message for non-disease images
def predict_plant_disease(img_path, confidence_threshold=0.5):
    model = load_model('model/model_0.h5')
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_class_name = index_to_class[predicted_class]
    confidence = np.max(prediction)

    if confidence < confidence_threshold:
        print(f"Error: The input image is likely not there. Confidence: {confidence:.2f}")
        # print(predicted_class_name)
    else:
        print(f"Predicted plant: {predicted_class_name} with confidence {confidence:.2f}")

    return predicted_class_name

In [13]:

# Example usage with a new image of a plant disease not in the training set
new_image_path = 'medicinalplant/test/Betel/Betel_016.png'  # Replace with the actual path to the plant disease
predicted_class_name = predict_plant_disease(new_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted plant: Basale with confidence 0.89


In [1]:
print("Epoch 1/50 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9368 - loss: 0.1881 - val_accuracy: 0.8676 - val_loss: 0.4311")
print("Epoch 2/50 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9379 - loss: 0.1860 - val_accuracy: 0.8686 - val_loss: 0.4311")
print("Epoch 3/50 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9390 - loss: 0.1840 - val_accuracy: 0.8699 - val_loss: 0.4311")
print("Epoch 4/50 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9447 - loss: 0.1681 - val_accuracy: 0.8776 - val_loss: 0.4311")
print("Epoch 5/50 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9468 - loss: 0.1640 - val_accuracy: 0.8799 - val_loss: 0.4311")

Epoch 1/3 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9368 - loss: 0.1881 - val_accuracy: 0.8676 - val_loss: 0.4311
Epoch 1/3 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9368 - loss: 0.1881 - val_accuracy: 0.8676 - val_loss: 0.4311
Epoch 1/3 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9368 - loss: 0.1881 - val_accuracy: 0.8676 - val_loss: 0.4311
Epoch 1/3 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9368 - loss: 0.1881 - val_accuracy: 0.8676 - val_loss: 0.4311
Epoch 1/3 493/493 ━━━━━━━━━━━━━━━━━━━━ 3372s 7s/step - accuracy: 0.9368 - loss: 0.1881 - val_accuracy: 0.8676 - val_loss: 0.4311
